# 월간 데이콘 법원 판결 예측 AI 경진대회
Dataset Info.
* ID : 사건 샘플 ID
* first_party : 사건의 첫 번째 당사자
* second_party : 사건의 두 번째 당사자
* facts : 사건 내용
* first_party_winner : 첫 번째 당사자의 승소 여부 (0 : 패배, 1 : 승리)

**첫 번째 당사자의 승소 여부 예측하기**

# **1. Importing libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import nltk # 문장 토크나이저

# 영어 불용어 - 불용어 모아 놓은 리스트 다운로드해 제거
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize # 토큰화
from nltk.stem.porter import PorterStemmer # 어근 동일화 <-> 이거 말고도 "Lancaster Stemmer"

# 표제어 추출
from nltk.stem import WordNetLemmatizer

# 정규표현 처리
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [ ]:
#
# 영어 데이터 전처리 함수
stops = set(stopwords.words('english'))
stemmer = nltk.stem.SnowballStemmer('english')

def cleaning(str):
    replaceAll = str

    # 특수문자 및 기호 등 필요없는 문자 제거
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)

    # 대소문자 모두 소문자로 통일
    no_capitals = only_english.lower().split()

    # 불용어(분석에 필요없는 토큰) 제거
    no_stops = [word for word in no_capitals if not word in stops]

    # 어근 추츨을 통한 텍스트 정규화 작업
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    return ' '.join(stemmer_words)

In [ ]:
# 데이터 늘린 걸로 한번해보자,,,ㅎㅎ
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/newtrain.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')

In [ ]:
print(train.duplicated(keep=False).sum())

0


# **2. 파생변수 생성**

* first_party_win_percent	: 첫 번째 당사자가 승소할 확률. 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
* test에서는 first_party가 train의 first_party와 겹치는 경우에는 해당 사람의 승소할 확률을 반영함. 겹치지 않는 경우는 0.5로 반영함.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
# winner: 해당 사건에서 승소한 사람
# frequency: winner의 총 등장 횟수
# win_percentage: 승소한 사람이 다른 사건에서도 승소할 확률

train['winner']=0
train['winner_frequency']=0
train['win_percentage']=0

for i in range(2477):
  if train['first_party_winner'][i]==1:
    train['winner'][i] = train['first_party'][i]
  else:
    train['winner'][i] = train['second_party'][i]

for i in range(2477):
  train['winner_frequency'][i] = len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]])
  train['win_percentage'][i]= len(train.loc[train['winner'] == train['winner'][i]]) /(len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]]))

In [ ]:
# train['loser']=0
# train['lose_frequency']=0
# train['lose_percentage']=0

# for i in range(2477):
#   if train['first_party_winner'][i]==0:
#     train['loser'][i] = train['first_party'][i]
#   else:
#     train['loser'][i] = train['second_party'][i]

# for i in range(2477):
#   train['lose_frequency'][i] = len(train.loc[train['first_party'] == train['loser'][i]]) + len(train.loc[train['second_party'] == train['loser'][i]])
#   train['lose_percentage'][i]= len(train.loc[train['loser'] == train['loser'][i]]) /(len(train.loc[train['first_party'] == train['loser'][i]]) + len(train.loc[train['second_party'] == train['loser'][i]]))

In [ ]:
# first_party_frequency: first_party가 나타난 빈도
# first_party_win_percent: first_party가 승소할 확률 / 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
train['first_party_win_percent'] = 0
train['first_party_frequency'] = 0

for i in range(2477):
  train['first_party_frequency'][i] = len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]])

  if (len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]])) > 1:
    train['first_party_win_percent'][i]= len(train.loc[train['winner'] == train['first_party'][i]]) /(len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]]))
  else:
    train['first_party_win_percent'][i] = 0.5

In [ ]:
#train=train.drop(['winner',	'winner_frequency',	'win_percentage',	'loser',	'lose_frequency',	'lose_percentage'],axis=1)
train=train.drop(['winner',	'winner_frequency',	'win_percentage'],axis=1)

In [ ]:
train_first_party = train[['first_party','first_party_win_percent']]

In [ ]:
train_first_party

,first_party,first_party_win_percent
0,Phil A. St. Amant,0.5
1,Stephen Duncan,0.5
2,Billy Joe Magwood,0.5
3,Linkletter,0.5
4,William Earl Fikes,0.5
...,...,...
3377,"Roman Catholic Diocese of Brooklyn, New York",0.0
3378,"Jose Santos Sanchez, et al.",0.0
3379,United States,0.0
3380,Florida,0.0


병합

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')
test_first_party = test['first_party']
test_first_party

0                                                 Salerno
1                  Milberg Weiss Bershad Hynes and Lerach
2       No. 07-582\t Title: \t Federal Communications ...
3                                         Harold Kaufman 
4                                                  Berger
                              ...                        
1235                Haitian Centers Council, Inc., et al.
1236                                              Whitman
1237                  Linda A. Matteo and John J. Madigan
1238        Washington State Apple Advertising Commission
1239                                     Theodore Stovall
Name: first_party, Length: 1240, dtype: object

In [ ]:
test_first_party_병합 = pd.merge(test_first_party, train_first_party, how='left')

In [ ]:
test_first_party_병합.isnull().sum()

first_party                  0
first_party_win_percent    461
dtype: int64

In [ ]:
test_first_party_병합 = test_first_party_병합.fillna(0.5)

In [ ]:
test_first_party_병합.head(20)

,first_party,first_party_win_percent
0,Salerno,0.500000
1,Milberg Weiss Bershad Hynes and Lerach,0.500000
2,No. 07-582\t Title: \t Federal Communications ...,0.000000
3,Harold Kaufman,0.000000
4,Berger,0.500000
5,Air Line Pilots Association,0.000000
6,Traffic Stream (BVI) Infrastructure Ltd.,0.500000
7,NRG Power Marketing LLC et al.,0.000000
8,United States,0.436782
9,United States,0.436782


#**3. Reading the dataset with pandas, dropping null values (calculated earlier at less than 5%) and creating a copy of the dataframe for the modelling process**

In [ ]:
df1 = train.copy()
df1.drop(columns=['first_party','second_party','first_party_win_percent','first_party_frequency'], inplace=True)
df1.head(2)

,facts,first_party_winner
0,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,Ramon Nelson was riding his bike when he suffe...,0


#**4. Seperating the dataset into target variables and two groups of independent variables, one (df_cat) which requires one-hot encoding to be machine readable and the other (df_nlp) which is text data which needs to be cleaned for features to be engineered from it.**

In [ ]:
df_target = df1['first_party_winner']
df_nlp = df1['facts']

# **5. Resetting indices to avoid NaNs during concatenation and performing one-hot encoding¶**

In [ ]:
df_target.reset_index(drop=True, inplace=True)
df_nlp.reset_index(drop=True, inplace=True)

In [ ]:
df_target1 = pd.DataFrame(df_target, columns=['first_party_winner'])
df_target1

,first_party_winner
0,1
1,0
2,1
3,0
4,1
...,...
3377,1
3378,0
3379,1
3380,0


In [ ]:
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])

In [ ]:
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)
df_nlp1

,facts
0,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,Ramon Nelson was riding his bike when he suffe...
2,An Alabama state court convicted Billy Joe Mag...
3,Victor Linkletter was convicted in state court...
4,"On April 24, 1953 in Selma, Alabama, an intrud..."
...,...
3377,In an effort to curb rising infections of COVI...
3378,Petitioners Jose Sanchez and his wife were cit...
3379,"Refugio Palomar-Santiago, a Mexican national, ..."
3380,This is an ongoing case of original jurisdicti...


# **6. Performing initial cleaning and tokenizing the corpus, introducing a function to perform further cleaning and Lemmatization upon the data.**

In [ ]:
corpus = df_nlp1["facts"]
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))

In [ ]:
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()

In [ ]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in
                    lst_stopwords]

    ## 어간추출 Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## 표제어 추출 Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
df_nlp1

,facts,facts_clean
0,"On June 27, 1962, Phil St. Amant, a candidate ...",june 27 1962 phil st amant candidate public of...
1,Ramon Nelson was riding his bike when he suffe...,ramon nelson riding bike suffered lethal blow ...
2,An Alabama state court convicted Billy Joe Mag...,alabama state court convicted billy joe magwoo...
3,Victor Linkletter was convicted in state court...,victor linkletter convicted state court eviden...
4,"On April 24, 1953 in Selma, Alabama, an intrud...",april 24 1953 selma alabama intruder broke apa...
...,...,...
3377,In an effort to curb rising infections of COVI...,effort curb rising infection covid19 new york ...
3378,Petitioners Jose Sanchez and his wife were cit...,petitioner jose sanchez wife citizen el salvad...
3379,"Refugio Palomar-Santiago, a Mexican national, ...",refugio palomarsantiago mexican national grant...
3380,This is an ongoing case of original jurisdicti...,ongoing case original jurisdiction fact explai...


# **8. Introducing Count Vectorizer to derive features from textual data.**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df_nlp2 = pd.concat([df_nlp1,df_target1['first_party_winner']],axis=1, join='inner')

In [ ]:
xfeatures = df_nlp2['facts_clean']
ylabel = df_nlp2['first_party_winner']

In [ ]:
xfeatures

0       june 27 1962 phil st amant candidate public of...
1       ramon nelson riding bike suffered lethal blow ...
2       alabama state court convicted billy joe magwoo...
3       victor linkletter convicted state court eviden...
4       april 24 1953 selma alabama intruder broke apa...
                              ...                        
3377    effort curb rising infection covid19 new york ...
3378    petitioner jose sanchez wife citizen el salvad...
3379    refugio palomarsantiago mexican national grant...
3380    ongoing case original jurisdiction fact explai...
3381    natural gas act nga 15 usc 717717z permit priv...
Name: facts_clean, Length: 3382, dtype: object

In [ ]:
ylabel

0       1
1       0
2       1
3       0
4       1
       ..
3377    1
3378    0
3379    1
3380    0
3381    1
Name: first_party_winner, Length: 3382, dtype: int64

#**9. Now including the one-hot encoded features within our model.**

In [ ]:
vectorizer=TfidfVectorizer()

def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts_clean'])
    else:
        X_facts = vectorizer.transform(df['facts_clean'])

    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])

    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [ ]:
frames = [train['first_party'],train['second_party'], xfeatures, train['first_party_winner']]
trains = pd.concat(frames, axis=1, join='inner')
trains


,first_party,second_party,facts_clean,first_party_winner
0,Phil A. St. Amant,Herman A. Thompson,june 27 1962 phil st amant candidate public of...,1
1,Stephen Duncan,Lawrence Owens,ramon nelson riding bike suffered lethal blow ...,0
2,Billy Joe Magwood,"Tony Patterson, Warden, et al.",alabama state court convicted billy joe magwoo...,1
3,Linkletter,Walker,victor linkletter convicted state court eviden...,0
4,William Earl Fikes,Alabama,april 24 1953 selma alabama intruder broke apa...,1
...,...,...,...,...
3377,"Roman Catholic Diocese of Brooklyn, New York","Andrew M. Cuomo, Governor of New York",effort curb rising infection covid19 new york ...,1
3378,"Jose Santos Sanchez, et al.","Alejandro N. Mayorkas, Secretary of Homeland S...",petitioner jose sanchez wife citizen el salvad...,0
3379,United States,Refugio Palomar-Santiago,refugio palomarsantiago mexican national grant...,1
3380,Florida,Georgia,ongoing case original jurisdiction fact explai...,0


In [ ]:
trains = trains.dropna(axis=0)
trains

,first_party,second_party,facts_clean,first_party_winner
0,Phil A. St. Amant,Herman A. Thompson,june 27 1962 phil st amant candidate public of...,1
1,Stephen Duncan,Lawrence Owens,ramon nelson riding bike suffered lethal blow ...,0
2,Billy Joe Magwood,"Tony Patterson, Warden, et al.",alabama state court convicted billy joe magwoo...,1
3,Linkletter,Walker,victor linkletter convicted state court eviden...,0
4,William Earl Fikes,Alabama,april 24 1953 selma alabama intruder broke apa...,1
...,...,...,...,...
3377,"Roman Catholic Diocese of Brooklyn, New York","Andrew M. Cuomo, Governor of New York",effort curb rising infection covid19 new york ...,1
3378,"Jose Santos Sanchez, et al.","Alejandro N. Mayorkas, Secretary of Homeland S...",petitioner jose sanchez wife citizen el salvad...,0
3379,United States,Refugio Palomar-Santiago,refugio palomarsantiago mexican national grant...,1
3380,Florida,Georgia,ongoing case original jurisdiction fact explai...,0


In [ ]:
# 벡터화 함수 적용
X_train_1 = get_vector(vectorizer,trains, True)
Y_train_1 = trains["first_party_winner"]

In [ ]:
x_train, x_test, yy_train, yy_test = train_test_split(X_train_1,Y_train_1, test_size=0.25)

#**Define Model & Train**

로지스틱 회귀

In [ ]:
#logistic
model = LogisticRegression()

X_train = np.asarray(x_train) # np.array 활용 및 적용
Y_train = np.asarray(yy_train)
X_test = np.asarray(x_test) # np.array 활용 및 적용
Y_test = np.asarray(yy_test)

model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
model.score(X_test,Y_test)

0.6307692307692307

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

랜덤포레스트

In [ ]:
#logistic

X_train = np.asarray(x_train) # np.array 활용 및 적용
Y_train = np.asarray(yy_train)
X_test = np.asarray(x_test) # np.array 활용 및 적용
Y_test = np.asarray(yy_test)

model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

In [ ]:
model.score(X_test,Y_test)

0.6497041420118344



```
# 코드로 형식 지정됨
```

KNeighborsClassifier(n_neighbors=3)

In [ ]:
X_train = np.asarray(x_train) # np.array 활용 및 적용
Y_train = np.asarray(yy_train)
X_test = np.asarray(x_test) # np.array 활용 및 적용
Y_test = np.asarray(yy_test)

model = knn=KNeighborsClassifier(n_neighbors=7)
model.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
model.score(X_test,Y_test)

0.6461538461538462

XGBClassifier()

In [ ]:
X_train = np.asarray(x_train) # np.array 활용 및 적용
Y_train = np.asarray(yy_train)
X_test = np.asarray(x_test) # np.array 활용 및 적용
Y_test = np.asarray(yy_test)

model = XGBClassifier()
model.fit(X_train, Y_train)

KeyboardInterrupt: ignored

In [ ]:
model.score(X_test,Y_test)

#**submission**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/sample_submission.csv')

In [ ]:
X_test = np.asarray(X_test)
pred = model.predict(X_test)

In [ ]:
submit['first_party_winner'] = pred
submit.to_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/code/submission/baseline_submit_logistic_newdata.csv', index=False)
print('Done')

Done
